In [47]:
from Men1.main import *
import pandas as pd
from sklearn import model_selection, datasets, linear_model, metrics, ensemble
from math import exp
import time

In [48]:
df = pd.read_csv("norm_data.csv")
df = df.to_numpy()
df=df.T
X = binominalization_0001(df[1])
X.astype('float32')
#X = X.T
print(X.shape)
y = df[2]
a = max(y)
y = y.astype('float32')
y = np.array(y+1)
y_st = np.log(y)
y_st = (y_st/y_st.std())
y_st = 4*(y_st - y_st.mean()) + 18
y = y_st


(29337, 144)


In [49]:
def AUCPR_ans(pred, y_curr, tau):
    return metrics.average_precision_score(y_curr < tau, pred < tau)


def SA_score(Lambda, train_size, tau):
    ss = model_selection.ShuffleSplit(n_splits = 20, test_size = 1 - train_size)

    met_AUCPR = []
    met_2 = []
    num = 1
    start_time_0 = time.process_time()
    for train_indices, test_indices in ss.split(X):
        start_time = time.process_time()
        train_data = X[train_indices]
        train_labels = y[train_indices]
        test_data = X[test_indices]
        test_labels = y[test_indices]
        print()
        print("-----------------")
        print()
        print(str(num)+" запуск")
        num += 1
        print("Создаём QUBO")
        Q = creating_QUBO(train_data, train_labels, Lambda)
        print("Обучаем")
        sampler = samplerSA()
        sampleset = sampler.sample_qubo(Q, num_reads=100, num_iter=50)
        SA_predictions = sampleset.predict(test_data)
        met_2.append(metrics.mean_squared_error(SA_predictions, test_labels))
        met_AUCPR.append(AUCPR_ans(SA_predictions, test_labels, tau))
        end_time = time.process_time()
        print()
        print("Этот запуск выполнился за " + str(round(end_time-start_time))+ " секунд")
        print("С начала запуска прошло " + str(round(end_time-start_time_0))+ " секунд")
    met_2 = np.array(met_2)
    met_AUCPR = np.array(met_AUCPR)
    return met_AUCPR.mean(), met_2.mean()

In [50]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = DWaveSampler(endpoint="https://cloud.dwavesys.com/sapi/", token="DEV-05e275d77bfe5870a5acefc67a233d5ef33ef1d7", solver="Advantage_system1.1")
sampler_embedded = EmbeddingComposite(sampler)
def DW_score(Lambda, train_size, tau):
    ss = model_selection.ShuffleSplit(n_splits = 20, test_size = 1 - train_size)

    met_AUCPR = []
    met_2 = []
    num = 1
    start_time_0 = time.process_time()
    for train_indices, test_indices in ss.split(X):
        start_time = time.process_time()
        train_data = X[train_indices]
        train_labels = y[train_indices]
        test_data = X[test_indices]
        test_labels = y[test_indices]
        print()
        print("-----------------")
        print()
        print(str(num)+" запуск")
        num += 1
        print("Создаём QUBO")
        Q = creating_QUBO(train_data, train_labels, Lambda)
        print("Обучаем")
        sampleset = sampler_embedded.sample_qubo(Q, num_reads=50)
        DW_predictions = sampleset.predict(test_data)
        met_2.append(metrics.mean_squared_error(DW_predictions, test_labels))
        met_AUCPR.append(AUCPR_ans(DW_predictions, test_labels, tau))
        end_time = time.process_time()
        print()
        print("Этот запуск выполнился за " + str(round(end_time-start_time))+ " секунд")
        print("С начала запуска прошло " + str(round(end_time-start_time_0))+ " секунд")
    met_2 = np.array(met_2)
    met_AUCPR = np.array(met_AUCPR)
    return met_AUCPR.mean(), met_2.mean()

In [ ]:
Q = creating_QUBO(X[:20], y[:20], 0.1)
sampleset = sampler_embedded.sample_qubo(Q, num_reads=50)
print(sampleset)
#print(((sampleset.data.to_numpy().T)[0:-2]).T)

In [45]:
predictDW(sampleset,X)

NameError: name 'predictDW' is not defined

In [14]:
ts = 0.01
Tau = 20
lambdas = [0.01,0.05,0.1,0.03,0.15,0.07]
SA = []
DW = []
for l in lambdas:
    SA.append(SA_score(l,ts, Tau))
    DW.append(DW_score(l,ts, Tau))

SA = np.array(SA)
DW = np.array(DW)


-----------------

1 запуск
Создаём QUBO
Обучаем
Запуск номер 10
Запуск номер 20
Запуск номер 30
Запуск номер 40
Запуск номер 50
Запуск номер 60
Запуск номер 70
Запуск номер 80
Запуск номер 90
Запуск номер 100

Этот запуск выполнился за 242 секунд
С начала запуска прошло 242 секунд

-----------------

2 запуск
Создаём QUBO
Обучаем
Запуск номер 10
Запуск номер 20
Запуск номер 30
Запуск номер 40
Запуск номер 50
Запуск номер 60
Запуск номер 70
Запуск номер 80
Запуск номер 90
Запуск номер 100

Этот запуск выполнился за 211 секунд
С начала запуска прошло 454 секунд

-----------------

3 запуск
Создаём QUBO
Обучаем
Запуск номер 10
Запуск номер 20
Запуск номер 30
Запуск номер 40
Запуск номер 50
Запуск номер 60
Запуск номер 70
Запуск номер 80
Запуск номер 90
Запуск номер 100

Этот запуск выполнился за 204 секунд
С начала запуска прошло 657 секунд

-----------------

4 запуск
Создаём QUBO
Обучаем
Запуск номер 10
Запуск номер 20
Запуск номер 30
Запуск номер 40
Запуск номер 50
Запуск номер 60
За

AttributeError: 'SampleSet' object has no attribute 'predict'

In [11]:
mer


array([[ 0.75690999, 18.664463  ],
       [ 0.75713773, 18.63350977],
       [ 0.75728098, 18.64638194]])